In [2]:
solveInt(1, 1) {
    readLines().sumOf {
        var firstDigit: Int = -1
        var lastDigit: Int = -1

        for (c in it) {
            if (c.isDigit()) {
                if (firstDigit == -1) firstDigit = c.digitToInt()
                lastDigit = c.digitToInt()
            }
        }

        firstDigit * 10 + lastDigit
    }
}

Day 1, part 1: 53386


In [8]:
val digitRegex = Regex("(\\d|(one)|(two)|(three)|(four)|(five)|(six)|(seven)|(eight)|(nine))")
val digitMap = mapOf(
    "one" to 1,
    "two" to 2,
    "three" to 3,
    "four" to 4,
    "five" to 5,
    "six" to 6,
    "seven" to 7,
    "eight" to 8,
    "nine" to 9,
)

solveInt(1, 2) {
    readLines().sumOf { line ->
        var firstDigit: Int = -1
        var lastDigit: Int = -1

        var cur = 0
        
        while (cur < line.length) {
            val match = digitRegex.find(line, cur)
            if (match == null) {
                ++cur
                continue
            }
            
            val target = match.groups[1] ?: continue
            
            ++cur
            
            val digit = digitMap[target.value] ?: target.value.toInt()
            if (firstDigit == -1) firstDigit = digit
            lastDigit = digit
        }
        val r = firstDigit * 10 + lastDigit

        //println("$r: $line")
        
        r
    }
}

Day 1, part 2: 53312


In [10]:
class D2_Round(val green: Int, val red: Int, val blue: Int)

solveInt(2, 1) {
    readLines().sumOf { line ->
        val (preamble, description) = line.split(": ")
        val (_, gameIdStr) = preamble.split(" ")
        val gameId = gameIdStr.toInt()
        
        val rounds = description.split("; ").map {roundStr ->
            val parts = roundStr.split(", ").associate {
                val (v, k) = it.split(" ")
                k to v.toInt()
            }
            
            D2_Round(parts["green"] ?: 0, parts["red"] ?: 0, parts["blue"] ?: 0)
        }
        
        if (rounds.all { it.red <= 12 && it.green <= 13 && it.blue <=14 }) gameId
        else 0
    }
}

Day 2, part 1: 2317


In [11]:
solveInt(2, 1) {
    readLines().sumOf { line ->
        val (preamble, description) = line.split(": ")
        val (_, gameIdStr) = preamble.split(" ")
        val gameId = gameIdStr.toInt()
        
        val rounds = description.split("; ").map {roundStr ->
            val parts = roundStr.split(", ").associate {
                val (v, k) = it.split(" ")
                k to v.toInt()
            }
            
            D2_Round(parts["green"] ?: 0, parts["red"] ?: 0, parts["blue"] ?: 0)
        }
        
        val powerRound = D2_Round(
            rounds.maxOf { it.green },
            rounds.maxOf { it.red },
            rounds.maxOf { it.blue }
        )
        
        with(powerRound) { green * red * blue }
    }
}

Day 2, part 1: 74804


In [1]:
solveInt(3, 1) {
    class NumRange(val num: Int, val line: Int, val rows: IntRange)
    val chars = mutableMapOf<Pair<Int, Int>, Char>()
    val nums = mutableListOf<NumRange>()
    
    readLines().forEachIndexed { i, line ->
        var curNum = 0
        var curNumStart = -1
        
        line.forEachIndexed { j, c ->
            when {
                c.isDigit() -> {
                    val d = c.digitToInt()
                    curNum = curNum * 10 + d
                    if (curNumStart == -1) curNumStart = j
                }
                else -> {
                    if (curNum > 0) {
                        nums.add(NumRange(curNum, i, curNumStart..(j - 1)))
                        curNum = 0
                        curNumStart = -1
                    }
                    if (c != '.') chars[i to j] = c
                }
            }
        }
        
        if (curNum > 0) {
            nums.add(NumRange(curNum, i, curNumStart..line.lastIndex))
        }
    }
    
    nums.filter { numRange ->
        sequence<Pair<Int, Int>> {
            val myRange = (numRange.rows.first - 1)..(numRange.rows.last + 1)
            
            for (j in myRange) {
                yield((numRange.line - 1) to j)
                yield((numRange.line + 1) to j)
            }
            yield(numRange.line to myRange.first)
            yield(numRange.line to myRange.last)
            
        }.any { chars.containsKey(it) }
    }.sumOf { it.num }
}

Day 3, part 1: 525181


In [3]:
solveInt(3, 2) {
    class NumRange(val num: Int, val line: Int, val rows: IntRange)
    val chars = mutableMapOf<Pair<Int, Int>, Char>()
    val nums = mutableMapOf<Pair<Int, Int>, NumRange>()
    
    readLines().forEachIndexed { i, line ->
        var curNum = 0
        var curNumStart = -1
        
        fun addNum(from: Int, to: Int) {
            val r = from..to
            val n = NumRange(curNum, i, r)
            for (j in r) nums[i to j] = n 
        }
        
        line.forEachIndexed { j, c ->
            when {
                c.isDigit() -> {
                    val d = c.digitToInt()
                    curNum = curNum * 10 + d
                    if (curNumStart == -1) curNumStart = j
                }
                else -> {
                    if (curNum > 0) {
                        addNum(curNumStart, (j - 1))
                        curNum = 0
                        curNumStart = -1
                    }
                    if (c != '.') chars[i to j] = c
                }
            }
        }
        
        if (curNum > 0) {
            addNum(curNumStart, line.lastIndex)
        }
    }
    
    chars.map { (coord, c) ->
        if (c == '*') {
            val allP = sequence<Pair<Int, Int>> {
                for (i in coord.first - 1 .. coord.first + 1) {
                    for (j in coord.second - 1 .. coord.second + 1) {
                        yield(i to j)
                    }
                }
            }.mapNotNull { nums[it] }.toSet()
            
            if (allP.size == 2) {
                allP.map { it.num }.reduce { acc, n -> acc * n }
            } else {
                0
            }
        } else {
            0
        }
    }.sum()
}

Day 3, part 2: 84289137
